#**Extração de Características Aurais**

#Preparação do ambiente do Notebook





**Instalando Dependências**

In [1]:
!pip install opencv-contrib-python==4.4.0.44
!pip install python_speech_features
!pip install speechRecognition

import cv2 as cv
import numpy as np
from python_speech_features import mfcc
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.audio.io.AudioFileClip import AudioFileClip
import pandas as pd
import pickle


     |████████████████████████████████| 55.7 MB 1.1 MB/s 
  Attempting uninstall: opencv-contrib-python
    Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30
  Created wheel for python-speech-features: filename=python_speech_features-0.6-py3-none-any.whl size=5888 sha256=a56a866cc152ccb10324b550eb359e951982ff0a4ea13f0fd9e61ac014387e91
  Stored in directory: /root/.cache/pip/wheels/b0/0e/94/28cd6afa3cd5998a63eef99fe31777acd7d758f59cf24839eb
Successfully built python-speech-features
     |████████████████████████████████| 32.8 MB 1.2 MB/s 


**Obtendo Dados**

In [2]:
! wget https://github.com/rudineigoularte/MBA_IABIGDATA/raw/main/videos2.tar.gz
! wget https://github.com/rudineigoularte/MBA_IABIGDATA/raw/main/videos3.tar.gz
! wget https://github.com/rudineigoularte/MBA_IABIGDATA/raw/main/videos4.tar.gz
! tar -xzvf videos2.tar.gz
! tar -xzvf videos3.tar.gz
! tar -xzvf videos4.tar.gz
! cp ./videos3/*.* ./videos2
! cp ./videos4/*.* ./videos2
! rm -r ./videos3
! rm -r ./videos4
! mkdir features

--2022-04-30 01:22:12--  https://github.com/rudineigoularte/MBA_IABIGDATA/raw/main/videos2.tar.gz
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rudineigoularte/MBA_IABIGDATA/main/videos2.tar.gz [following]
--2022-04-30 01:22:13--  https://raw.githubusercontent.com/rudineigoularte/MBA_IABIGDATA/main/videos2.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67222812 (64M) [application/octet-stream]
Saving to: ‘videos2.tar.gz’

videos2.tar.gz      100%[===================>]  64.11M   409MB/s    in 0.2s    

2022-04-30 01:22:13 (409 MB/s) - ‘videos2.tar.gz’ saved [67222812/67222812]

--2022-0

**Acesso ao Drive - para armazenamento de Arquivos**

O *pathname* precisa ser personalizado com o Drive do usuário deste Notebook. Atenção: alguns *pathnames* podem estar em Inglês enquanto outros em Português - verifique o seu caso. Crie um pasta no **seu** Drive onde os arquivos serão armazenados. Neste exemplo tal pasta é "MBA_Curso14_Dados".

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [4]:
cd '/content/drive/My Drive/MBA_Curso14_Dados/'

/content/drive/My Drive/MBA_Curso14_Dados


#<h1>Função para Extração das Features Aurais</h1>

In [5]:
## ==========
# Pré-processamento dos dados: 
# Redução de ruído: elimina-se os 10s iniciais e finais de cada vídeo (vinhetas e aberturas). 
# Redução de instâncias/características: Para a extração de característica aurais 
# não se realizou a redução de instâncias/características. Os vídeos da base utilizada no 
# curso são bastante curtos e com poucas "falas" em geral. O que produz poucos
# dados em comparação às features visuais. Optou-se, nesse caso, por usar o vídeo
# todo para a extração das features aurais.
#
# A variável de retorno "features" é um dicionário de tamanho n, com n sendo a 
# quantidade de vídeos. Cada posição do dicionário representa um vídeo x, e contém
# uma lista ("bag") que, por sua vez, contém os vetores de caracterísiticas MFCC
# do vídeo x inteiro.
## ==========
def extractMfcc(videos):
    print("Extraindo caracterísitcas MFCC...")
    features = dict()
    bag = [] #usado para manter compatibilidade com getBagOfFeautures no caso visual

    i = 0
    for videoName in videos[0]: 
      print(videoName)   
      video = VideoFileClip("/content/videos2/"+videoName)
      #fps = video.fps
      duration = video.duration
      video.close()

      audio = AudioFileClip("/content/videos2/"+videoName)
      
      #Pula-se os 10s iniciais e finais de cada vídeo - vinhetas/aberturas/créditos
      audioClip = audio.subclip(10, duration-10).to_soundarray() 
      audio.close()
      
      bag.append(mfcc(audioClip, 44100, nfft=2048))
      features[i] = bag #compatibilidade com caso visual
      i = i + 1
      bag = []
    return features

#Script para extrair características aurais

In [6]:

#Preparação dos dados
videos = pd.read_csv('/content/videos2/videos2.csv', sep = ',', header = None)

#Extração de características MFCC
auralFeatures = extractMfcc(videos)
#print("Tamanho auralFeatures: ",len(auralFeatures))

#Gravando as featrures em arquivo
#A pasta "features" deve existir dentro de "MBA_Curso14_Dados"
with open('./features2/aural_features.arq', 'wb') as arq:
    pickle.dump(auralFeatures, arq)

Extraindo caracterísitcas MFCC...
1801.mp4
54176.mp4
55274.mp4
66915.mp4
71264.mp4
79333.mp4
79925.mp4
80285.mp4
93128.mp4
131239.mp4
